<a href="https://colab.research.google.com/github/Machine-Learning-for-Finance/Machine-Learning-Algorithms/blob/master/01-Data%20Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sklearn.datasets
import numpy as np
import matplotlib.pyplot as plt
from os import environ

environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"

import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils import hello_world
hello_world()

# Loading From Google Drive

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base_location = "/content/drive/My Drive/data/"
except:
    base_location = "/data/FannieMae/"

In [3]:
!ls /content/drive/My\ Drive/data/2010Q1

ls: cannot access '/content/drive/My Drive/data/2010Q1': No such file or directory


# Financial Datasets

There are a large number of financial datasets that are available, the first one we will discuss is the Fannie Mae Fixed Rate Mortgage Dataset.

- https://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html

- https://loanperformancedata.fanniemae.com/lppub/index.html#Portfolio

Which consists of both Acquisitions and Performance data for a collection of mortgages.

To make our lives a bit easier here are the column names pulled from the provided `R` script to load the data.  See `R` is good for at least one thing.

In [4]:
AcquisitionColumnNames = (
    "LOAN_ID", "ORIG_CHN", "Seller.Name", 
    "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE",
    "FRST_DTE", "OLTV", "OCLTV", "NUM_BO", 
    "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", 
    "PROP_TYP", "NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3", 
    "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", 
    "RELOCATION_FLG"
)

PerformanceColumnNames = (
    "LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", 
    "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat", 
    "Adj.Month.To.Mat", "Maturity.Date", "MSA", 
    "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", 
    "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", 
    "FCC_COST", "PP_COST", "AR_COST", "IE_COST", 
    "TAX_COST", "NS_PROCS","CE_PROCS", "RMW_PROCS", 
    "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", 
    "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"
)

In [5]:
# Data path will change depending on your system setup.

acquisition_data_path = f"{base_location}2010Q1/Acquisition_2010Q1.txt"

#### Loading the Acquisition Data from CSV

To load the data we call from `pandas`, `pd.read_csv` which automatically handles loading data from the csv file.  We provide column names, a notification that the file doesn't include headers, and information on what the column separator is `|` in this case.

In [6]:
acquisition_df = pd.read_csv(
    acquisition_data_path,
    names=AcquisitionColumnNames,
    header=None,
    sep="|"
)

In [7]:
acquisition_df.columns

Index(['LOAN_ID', 'ORIG_CHN', 'Seller.Name', 'ORIG_RT', 'ORIG_AMT', 'ORIG_TRM',
       'ORIG_DTE', 'FRST_DTE', 'OLTV', 'OCLTV', 'NUM_BO', 'DTI', 'CSCORE_B',
       'FTHB_FLG', 'PURPOSE', 'PROP_TYP', 'NUM_UNIT', 'OCC_STAT', 'STATE',
       'ZIP_3', 'MI_PCT', 'Product.Type', 'CSCORE_C', 'MI_TYPE',
       'RELOCATION_FLG'],
      dtype='object')

#### Loading the Performance Data

In [8]:
performance_data_path = f"{base_location}2010Q1/Performance_2010Q1.txt"

performance_df = pd.read_csv(
    performance_data_path,
    names=PerformanceColumnNames,
    header=None,
    sep="|"
)

/home/capcolabs/Stevens/Machine-Learning-Algorithms/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


First thing we note is that this takes longer than the `Acquisitions` data to load, stemming from the fact that for each loan there are multiple monthly data elements loaded.

# Data Modifying

In [9]:
DS = set(performance_df['Delq.Status'])

In [10]:
print(DS)
mapper = {}
for ds in DS:
    try:
        mapper[ds] = int(ds)
    except:
        mapper[ds] = -1

performance_df['Delq.Status'] = performance_df['Delq.Status'].map(mapper)

{nan, '40', '55', '64', '27', '24', '28', '88', '89', '52', '87', '42', '53', '56', '67', '99', '57', '86', '49', '93', '36', '90', '43', '20', '26', '74', '9', '84', '75', '66', '95', '17', '41', '37', '11', '25', '58', '31', '50', '81', '5', '63', '71', '97', '47', '35', '38', '85', '65', '16', '62', '70', '32', '18', '29', '83', '23', '12', '14', '69', '82', '79', '4', '10', '54', '19', '91', '45', '46', '92', '34', '94', '22', '78', '33', '2', '72', '30', '15', '61', '76', 'X', '98', '0', '1', '80', '73', '77', '96', '59', '8', '44', '51', '48', '21', '68', '7', '13', '6', '39', '60', '3'}


In [11]:
V, C = np.unique(performance_df['Delq.Status'], return_counts=True)
print(V)

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70
 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94
 95 96 97 98 99]


# Grouping By Loan ID

https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-value-in-groups-using-groupby

In [12]:
loans = performance_df.groupby("LOAN_ID", sort=True)['Delq.Status'].max()

ID_To_Delinq = {}

for row in loans.iteritems():
    loan_id, delinq = row
    ID_To_Delinq[loan_id] = delinq

In [13]:
def mapper(row):
    return ID_To_Delinq.get(row["LOAN_ID"], -1)

acquisition_df['MAX_DELINQ'] = acquisition_df.apply(mapper, axis=1)

In [14]:
V, C = np.unique(acquisition_df['MAX_DELINQ'], return_counts=True)

# Getting Forclousure Status

In [71]:
FCC_DTE = performance_df['FCC_DTE'].notna()

forclosed = performance_df[FCC_DTE]

FORECLOSURES = {}

for row in forclosed.iterrows():
    row = row[1]
    FORECLOSURES[row['LOAN_ID']] = row['FCC_DTE']

FORCLOSED = set(forclosed['LOAN_ID'])

def mapper(row):
    # return FORECLOSURES.get(row['LOAN_ID'], "NO_FCC")
    return int(row['LOAN_ID'] in FORCLOSED)

acquisition_df['FCC'] = acquisition_df.apply(mapper, axis=1)

In [72]:
print(set(acquisition_df['FCC']))

{0, 1}


## Getting Delinquency Status

In [73]:
V, C = np.unique(
    performance_df['Monthly.Rpt.Prd'], 
    return_counts=True
)

In [74]:
actual_date = performance_df['Monthly.Rpt.Prd'] == "01/01/2015"
next_date   = performance_df['Monthly.Rpt.Prd'] == "01/01/2016"

date_df = performance_df[actual_date]
next_df = performance_df[next_date]

Delinquency = {}
Next_Delinquency = {}

for row in date_df.iterrows():
    row = row[1]
    Delinquency[row['LOAN_ID']] = ID_To_Delinq.get(row["LOAN_ID"], -1)
    
for row in next_df.iterrows():
    row = row[1]
    Next_Delinquency[row['LOAN_ID']] = ID_To_Delinq.get(row['LOAN_ID'], -1)

In [75]:
def mapper(row):
    return Delinquency.get(row["LOAN_ID"], -1)

def next_mapper(row):
    return Next_Delinquency.get(row['LOAN_ID'], -1)

acquisition_df['DELINQ_DATE'] = acquisition_df.apply(mapper, axis=1)
acquisition_df['DELINQ_NEXT'] = acquisition_df.apply(next_mapper, axis=1)

In [76]:
delinq = acquisition_df['DELINQ_DATE'] > 0
delinq_df = acquisition_df[delinq]

print(f"{len(delinq_df.index)}")

def check_date_range(row):
    return row['DELINQ_NEXT'] >= row['DELINQ_DATE']

delinq_df['DELINQ_DELTA'] = delinq_df.apply(
    check_date_range,
    axis=1
)

20303


/home/capcolabs/Stevens/Machine-Learning-Algorithms/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [77]:
V, C = np.unique(delinq_df['DELINQ_DELTA'], return_counts=True)

for v, c in zip(V, C):
    print(v, " | ", c)

False  |  2219
True  |  18084


In [78]:
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
import seaborn as sns


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           #xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [79]:
df = delinq_df

DEL_NOTNAN = df["DELINQ_DELTA"].notna()
df = df[DEL_NOTNAN]
OLTV = df['OLTV'].notna()
df = df[OLTV]
CS = df['CSCORE_B'].notna()
df = df[CS]
DTI = df['DTI'].notna()
df = df[DTI]

credit_score  = np.array(df['CSCORE_B'])
loan_to_value = np.array(df['OLTV'])
debt_to_income= np.array(df['DTI'])
delinq_value  = np.array(df['DELINQ_DATE'])

max_delinq    = np.array(df['DELINQ_DELTA'])
foreclosed    = np.array(df['FCC'])

X = np.array(
    [
        credit_score, 
        loan_to_value, 
        debt_to_income, 
        delinq_value
    ]
).transpose()


y = np.array([foreclosed]).transpose()
print(X.shape)
print(y.shape)

Total = np.hstack([X, y])
print(Total.shape)
np.random.shuffle(Total)

X = Total[:, :4]
y = Total[:, 4:]

print(X.shape)
print(y.shape)

prop = 0.9
train_num = int(prop * len(Total))
print(f"Train Number: {train_num}")

X_train, X_test = X[:train_num], X[train_num:]
y_train, y_test = y[:train_num], y[train_num:]

print(f"X_Train: {X_train.shape}")
print(f"X_Test: {X_test.shape}")
print("=="*10)
print(f"y_Train: {y_train.shape}")
print(f"y_Test:  {y_test.shape}")

V, C = np.unique(y, return_counts=True)

class_weight = {}
for v, c in zip(V, C):
    prop = c / len(y)
    class_weight[v] = 1 - prop
    print(v, " | ", c)

class_names = np.unique(y)

print(class_names)

(20038, 4)
(20038, 1)
(20038, 5)
(20038, 4)
(20038, 1)
Train Number: 18034
X_Train: (18034, 4)
X_Test: (2004, 4)
y_Train: (18034, 1)
y_Test:  (2004, 1)
0.0  |  19397
1.0  |  641
[0. 1.]
